In [17]:
import pandas as pd 
import chromadb
import os

chroma_path ="../chroma_db"

df = pd.read_pickle("../data/embedded_data.pkl")



In [18]:
df["id"] = "tweet" + df.index.astype(str)
df.head()

,airline_sentiment,cleaned_text,embedding,id
0,negative,2293 cancelled flightled one,"[0.013153999112546444, 0.005188405979424715, -...",tweet0
1,positive,joanna wonderful job thank,"[-0.03579651564359665, 0.0400991216301918, 0.0...",tweet1
2,neutral,policy behind alter passenger seating assignme...,"[0.027994729578495026, 0.02059757336974144, -0...",tweet2
3,neutral,set figured luggage office,"[-0.030090009793639183, 0.015454476699233055, ...",tweet3
4,negative,im get plane think deserve know mechanical iss...,"[-0.025789836421608925, 0.024426547810435295, ...",tweet4


<h1 style="color:orange;"> Creating collection ! ! </h1>



In [19]:
client = chromadb.PersistentClient(path=chroma_path)

try:
    client.delete_collection(name="tweets")
    print("the collectios was deleted")
except:
    print("pass")
    pass

# metadata={"hnsw:space": "cosine"} tells Chroma to use Cosine Similarity for search

collection = client.create_collection(name="tweets", metadata={"hnsw:space": "cosine"})


the collectios was deleted


In [22]:
# We will use a loop (batching) to avoid crashing the computer. We map the DataFrame columns to the 3 ChromaDB parts (ids, embeddings, metadatas)

from tqdm import tqdm

def add_to_collection(collection, df, batch_size=2000):
    total_rows = len(df)
    # Loop from 0 to total_rows in steps of 2000
    embeddings = list()
    for i in tqdm(range(0, total_rows, batch_size),desc=f'add to {collection.name}'):
        
        batch = df.iloc[i : i+ batch_size]
        
        ids = batch['id'].tolist()
        
        embeddings = batch["embedding"].tolist()
        
        metadatas = batch[["airline_sentiment", "cleaned_text"]].to_dict("records")
        
        collection.add(
            ids=ids,
            embeddings=embeddings,
            metadatas=metadatas
        )

In [21]:
print("Starting ingestion for Data...")

add_to_collection(collection, df)

Starting ingestion for Data...


add to tweets: 100%|██████████| 14/14 [00:06<00:00,  2.15it/s]
